In [1]:
import numpy as np, os, h5py
import cupy as cp
import matplotlib.pyplot as plt

import astropy.units as u
import astropy.constants as cst

from glob import glob
from tqdm import tqdm
from moviepy.editor import ImageSequenceClip

from create_ic_solarsystem import solar_system

Solar System data saved to 'solar_system.hdf5'


In [2]:
def get_accelleration(pos, mass, G, softening):
    """ Calculate the acceleration on each particle due to Newton's Law 
        
        Inputs:
            - pos (ndarray): array position of the particles with shape (N,3)
            - mass (ndarray): array mass of the particle with shape (N,1)
            - G (float): Newton Gravitational constant in the same units as the velocity and postion of the particles
            - softening (float): the softening length to avoid numerical infinity
        Return:
            - a (ndarray): array of the accelleration of the particles with shape (N,3)
    """
    # positions r = [x,y,z] for all particles
    r_x = pos[:,0:1]
    r_y = pos[:,1:2]
    r_z = pos[:,2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    rdx = r_x.T - r_x
    rdy = r_y.T - r_y
    rdz = r_z.T - r_z
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (rdx**2 + rdy**2 + rdz**2 + softening)**(-1.5)
    
    a_x = G * (rdx * inv_r3) @ mass
    a_y = G * (rdy * inv_r3) @ mass
    a_z = G * (rdz * inv_r3) @ mass
    
    # pack together the acceleration components
    #a = np.vstack((a_x, a_y, a_z))
    a = np.array([a_x, a_y, a_z]).T

    return a

# N-body simulation

Simulation parameters and constants

In [17]:
# time start, time-step and time end
t_start, t_end, dt = (0.0*u.day).value, (10*365.0*u.day).value, (1.0*u.day).value

# softening length
softening = (1e-12 *u.AU).value

# Newton's Gravitational constant
cst.G.to('AU^3/(Msun*day^2)').value

0.00029591220819207774

In [8]:
# Constants
#particle_names = solar_system.keys()
particle_names = ['Sun', 'Earth', 'Jupiter', "Halley's Comet"]

mass = np.array([solar_system[key]['mass'] for key in particle_names]) #* u.Msun
pos = np.array([solar_system[key]['position'] for key in particle_names]) #* u.AU
vel = np.array([solar_system[key]['velocity'] for key in particle_names]) #* u.AU/u.day

# Number of particles
N = pos.shape[0]

print(particle_names, pos.shape)
print(mass)
print(pos)
print(vel)

['Sun', 'Earth', 'Jupiter', "Halley's Comet"] (4, 3)
[1.000e+00 3.003e-06 9.545e-04 1.106e-16]
[[0.    0.    0.   ]
 [1.    0.    0.   ]
 [5.202 0.    0.   ]
 [0.586 0.    0.   ]]
[[0.         0.         0.        ]
 [0.         0.01719795 0.        ]
 [0.         0.00756525 0.        ]
 [0.         0.0314853  0.        ]]


In [9]:
# calculate initial gravitational accelerations
acc = get_accelleration(pos, mass, G_grav, softening)

# number of timesteps
Nt = int(np.ceil(t_end/dt))

# save particle orbits
pos_save = np.zeros((N,3,Nt+1))
pos_save[:,:,0] = pos

t_all = np.arange(Nt+1)*dt
t = t_start

# Simulation Main Loop
for i in tqdm(range(Nt)):
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt

    # update accelerations
    acc = get_accelleration(pos, mass, cst.G.to('AU^3/(Msun*day^2)').value, softening)

    # (1/2) kick
    vel += acc * dt/2.0

    # update time
    t += dt

    # save energies, positions for plotting trail
    pos_save[:,:,i+1] = pos

100%|█████████████████████████████████████| 3650/3650 [00:00<00:00, 6748.19it/s]


## Plot

In [16]:
path_out = './plot_solarsystem/'
i_start = len(glob(path_out+'snapshot_*.png'))

xyz_scale = 5
#xyz_scale = 1150

for i in tqdm(range(i_start, Nt+1)):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection='3d')
    
    ax.grid(True, color='grey', alpha=0.2)  # Grid lines with transparency
    ax.set_facecolor('white')
    
    x, y, z = pos_save[...,i].T
    ax.scatter(x, y, z, marker='o', s=20, color='tab:blue')
    
    if(i < 25):
        ax.scatter(pos_save[:,0,:i].T, pos_save[:,1,:i].T, pos_save[:,2,:i].T, marker='o', s=5, color='tab:orange', alpha=0.2)
    else:
        ax.scatter(pos_save[:,0,i-25:i].T, pos_save[:,1,i-25:i].T, pos_save[:,2,i-25:i].T, marker='o', s=5, color='tab:orange', alpha=0.2)
    
    pos_cm = np.mean(pos_save[...,i]*mass[...,None], axis=0)/np.mean(mass)
    ax.set_xlim(pos_cm[0]-xyz_scale, pos_cm[0]+xyz_scale), ax.set_ylim(pos_cm[1]-xyz_scale, pos_cm[1]+xyz_scale), ax.set_zlim(pos_cm[2]-xyz_scale, pos_cm[2]+xyz_scale)
    
    ax.set_xlabel('x [AU]'), ax.set_ylabel('y [AU]'), ax.set_zlabel('z [AU]')
    
    plt.savefig(path_out+'snapshot_%d.png' %i, bbox_inches='tight')
    plt.clf(), plt.close()

100%|███████████████████████████████████████| 3651/3651 [07:16<00:00,  8.37it/s]


## Create GIF

In [18]:
def CreateMovie(filename, array, fps=5, scale=1., fmt='avi'):
    ''' Create and save a gif or video from array of images.
        Parameters:
            * filename (string): name of the saved video
            * array (list or string): array of images name already in order, if string it supposed to be the first part of the images name (before iteration integer)
            * fps = 5 (integer): frame per seconds (limit human eye ~ 15)
            * scale = 1. (float): ratio factor to scale image hight and width
            * fmt (string): file extention of the gif/video (e.g: 'gif', 'mp4' or 'avi')
        Return:
            * moviepy clip object
    '''
    if(isinstance(array, str)):
        array = np.array(sorted(glob(array+'*.png'), key=os.path.getmtime))
    else:
        pass
    filename += '.'+fmt
    clip = ImageSequenceClip(list(array), fps=fps).resize(scale)
    if(fmt == 'gif'):
        clip.write_gif(filename, fps=fps)
    elif(fmt == 'mp4'):
        clip.write_videofile(filename, fps=fps, codec='mpeg4')
    elif(fmt == 'avi'):
        clip.write_videofile(filename, fps=fps, codec='png')
    else:
        print('Error! Wrong File extension.')
        sys.exit()

    # print the size of the movie
    command = os.popen('du -sh %s' % filename)
    print(command.read())
    return clip


In [19]:
img_arr = ['%ssnapshot_%d.png' %(path_out, i) for i in range(len(glob(path_out+'*png')))]

#CreateMovie(filename='earth_sun_system', array=img_arr, fps=15, fmt='avi')
CreateMovie(filename='solar_system', array=img_arr, fps=15, fmt='avi')

Moviepy - Building video solar_system.avi.
Moviepy - Writing video solar_system.avi



Moviepy - Done !
Moviepy - video ready solar_system.avi
270M	solar_system.avi



In [21]:
pos_save[3,:,-1]

array([-20.4855863 ,   4.08417744,   0.        ])